In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.impute import SimpleImputer

# Load dataset
file_path = '/mnt/data/loan_data.csv'
data = pd.read_csv(file_path)

# Basic inspection
print(data.head())
print(data.info())

# Handle missing values
imputer = SimpleImputer(strategy='mean')  # Replace 'mean' with 'most_frequent' for categorical variables
data.fillna(method='ffill', inplace=True)

# Encode categorical variables
label_encoders = {}
for col in data.select_dtypes(include='object').columns:
    le = LabelEncoder()
    data[col] = le.fit_transform(data[col])
    label_encoders[col] = le

# Scale numerical features
scaler = StandardScaler()
numerical_cols = data.select_dtypes(include='number').columns
data[numerical_cols] = scaler.fit_transform(data[numerical_cols])

# Split the data
# Replace 'loan_status' with the actual target column name from your dataset
X = data.drop('loan_status', axis=1)
y = data['loan_status']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

   person_age person_gender person_education  person_income  person_emp_exp  \
0        22.0        female           Master        71948.0               0   
1        21.0        female      High School        12282.0               0   
2        25.0        female      High School        12438.0               3   
3        23.0        female         Bachelor        79753.0               0   
4        24.0          male           Master        66135.0               1   

  person_home_ownership  loan_amnt loan_intent  loan_int_rate  \
0                  RENT    35000.0    PERSONAL          16.02   
1                   OWN     1000.0   EDUCATION          11.14   
2              MORTGAGE     5500.0     MEDICAL          12.87   
3                  RENT    35000.0     MEDICAL          15.23   
4                  RENT    35000.0     MEDICAL          14.27   

   loan_percent_income  cb_person_cred_hist_length  credit_score  \
0                 0.49                         3.0           561  

<ipython-input-1-d04ae829e78c>:16: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data.fillna(method='ffill', inplace=True)


In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import StackingClassifier



# Load dataset
file_path = '/mnt/data/loan_data.csv'
import pandas as pd
data = pd.read_csv(file_path)

In [4]:
!pip install lime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.7/275.7 kB 5.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for lime: filename=lime-0.2.0.1-py3-none-any.whl size=283834 sha256=4c7daacc7fe56a35bc392881e282822455345e98afa1eb4435a1b638f039ed72
  Stored in directory: /root/.cache/pip/wheels/fd/a2/af/9ac0a1a85a27f314a06b39e1f492bee1547d52549a4606ed89
Successfully built lime


In [2]:
# Install 'lime' directly within the current cell
!pip install lime

import shap
import lime.lime_tabular
from sklearn.ensemble import RandomForestRegressor # import RandomForestRegressor instead of RandomForestClassifier

# ... rest of your code ...

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.7/275.7 kB 5.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for lime: filename=lime-0.2.0.1-py3-none-any.whl size=283834 sha256=9c3a0f39cf11cf97fb7e88ba8b68c8b2654b7c4f130cf982fdbb201db6383257
  Stored in directory: /root/.cache/pip/wheels/fd/a2/af/9ac0a1a85a27f314a06b39e1f492bee1547d52549a4606ed89
Successfully built lime


In [6]:
import shap
import lime.lime_tabular
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor # import RandomForestRegressor instead of RandomForestClassifier

# Load dataset (Assuming your data is in 'loan_data.csv')
# **Check if the file exists at this path or if the path is correct.**
file_path = '/mnt/data/loan_data.csv'  # **Verify this path!**
# **If the file is in the same directory as your notebook, you can use just the filename:**
# file_path = 'loan_data.csv'
# **If the file is in a different location, provide the full, correct path.**
# **Example: file_path = '/home/user/data/loan_data.csv'**

try:
    data = pd.read_csv(file_path)
except FileNotFoundError:
    print(f"Error: File not found at '{file_path}'. Please check the file path.")
    # Handle the error by setting data to None to avoid the NameError
    data = None
    # You can add more robust error handling, like exiting or using a default dataset
    # Example:
    # import sys
    # sys.exit(1)  # Exit with an error code
    # Or:
    # data = pd.DataFrame(...) # Create an empty or default DataFrame


# Check if data was successfully loaded
if data is not None:
    # Assuming 'Loan_Status' or a similar column is your target variable
    # Replace 'Loan_Status' with your actual target column name
    target_column = 'Loan_Status'

    # Separate features (X) and target (y)
    X = data.drop(columns=[target_column])
    y = data[target_column]

    # Split data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42) # Adjust test_size and random_state as needed

    # Initialize the RandomForestRegressor model
    rf_model = RandomForestRegressor(random_state=42)  # Create an instance of the model

    # SHAP Analysis
    rf_model.fit(X_train, y_train)
    explainer = shap.TreeExplainer(rf_model)
    shap_values = explainer.shap_values(X_test)
    shap.summary_plot(shap_values, X_test, feature_names=X_test.columns)

    # LIME Analysis
    # For regression tasks, 'class_names' is not applicable.
    lime_explainer = lime.lime_tabular.LimeTabularExplainer(
        training_data=X_train.values,
        feature_names=X_train.columns,
        mode='regression'  # Change mode to 'regression'
    )
    lime_exp = lime_explainer.explain_instance(X_test.iloc[0].values, rf_model.predict)
    lime_exp.show_in_notebook()
else:
    print("Error: Could not proceed with analysis as data loading failed.")

Error: File not found at '/mnt/data/loan_data.csv'. Please check the file path.
Error: Could not proceed with analysis as data loading failed.
